In [ ]:
!pip install -q ray recbole==1.2.0 kmeans-pytorch
!pip uninstall -y numpy && pip install -q "numpy<2"
print("✓ Done! Now restart: Runtime -> Restart session")

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy

In [ ]:
import torch
import torch.nn as nn
import torch.distributed as dist
import numpy as np
import json
import os
from scipy import stats
from recbole.quick_start import run_recbole
from recbole.model.general_recommender.neumf import NeuMF
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.trainer import Trainer
from recbole.utils import init_seed

# ============================================
# ONE-TIME PATCHES (safe to re-run)
# ============================================
if not hasattr(dist, '_barrier_patched'):
    _original_barrier = dist.barrier
    def _patched_barrier(*args, **kwargs):
        if dist.is_available() and dist.is_initialized():
            return _original_barrier(*args, **kwargs)
    dist.barrier = _patched_barrier
    dist._barrier_patched = True
    print("✓ Applied distributed patch")

if not hasattr(torch, '_load_patched'):
    _original_torch_load = torch.load
    def _patched_torch_load(*args, **kwargs):
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return _original_torch_load(*args, **kwargs)
    torch.load = _patched_torch_load
    torch._load_patched = True
    print("✓ Applied torch.load patch")

# ============================================
# CONFIGURATION
# ============================================
device = torch.device('cuda')
print(f"Using: {torch.cuda.get_device_name(0)}")

DATASET = 'ml-100k'
RATIO = 50
SEEDS = list(range(5))
CHECKPOINT_FILE = 'checkpoint_results.json'

def get_config():
    return {
        'model': 'NeuMF',
        'dataset': DATASET,
        'config_file_list': [],
        'data_path': './dataset',
        'neg_sampling': {'uniform': RATIO},
        'epochs': 100,
        'train_batch_size': 256,
        'eval_batch_size': 256,
        'learning_rate': 0.001,
        'mf_embedding_size': 64,
        'mlp_embedding_size': 64,
        'mlp_hidden_size': [128, 64, 32],
        'dropout_prob': 0.0,
        'eval_args': {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full'},
        'metrics': ['Hit', 'NDCG'],
        'topk': [5, 10, 20],
        'valid_metric': 'NDCG@10',
        'stopping_step': 10,
        'checkpoint_dir': 'saved/',
        'show_progress': True,
        'device': device,
    }

# ============================================
# LOAD CHECKPOINT OR START FRESH
# ============================================
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, 'r') as f:
        results = json.load(f)
    print(f"✓ Loaded checkpoint: BCE={len(results.get('bce',[]))} seeds done")
else:
    results = {'bce': [], 'focal': [], 'alpha_bce': []}
    print("Starting fresh...")

def save_checkpoint():
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(results, f)
    print(f"  → Checkpoint saved!")

# ============================================
# RUN BCE (with resume)
# ============================================
print("\n" + "="*60)
print("Running BCE...")
print("="*60)

start_seed = len(results['bce'])
for i, seed in enumerate(SEEDS[start_seed:], start=start_seed):
    print(f"\nSeed {seed} ({i+1}/10)...")
    cfg = get_config()
    cfg['seed'] = seed
    result = run_recbole(model='NeuMF', dataset=DATASET, config_dict=cfg)
    results['bce'].append(float(result['test_result']['ndcg@10']))
    save_checkpoint()

print(f"\n✓ BCE complete: {np.mean(results['bce']):.4f} ± {np.std(results['bce']):.4f}")

Using: Tesla T4
✓ Loaded checkpoint: BCE=1 seeds done

Running BCE...

Seed 1 (2/10)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

Seed 2 (3/10)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

Seed 3 (4/10)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

Seed 4 (5/10)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

✓ BCE complete: 0.0580 ± 0.0030


In [ ]:
# Focal Loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, logits, labels):
        probs = torch.sigmoid(logits)
        ce_loss = nn.functional.binary_cross_entropy_with_logits(logits, labels, reduction='none')
        pt = torch.where(labels == 1, probs, 1 - probs)
        alpha_t = torch.where(labels == 1, self.alpha, 1 - self.alpha)
        focal_weight = alpha_t * (1 - pt) ** self.gamma
        return (focal_weight * ce_loss).mean()

class NeuMFFocal(NeuMF):
    def __init__(self, config, dataset, gamma=2.0, alpha=0.25):
        super().__init__(config, dataset)
        self.focal_loss = FocalLoss(gamma, alpha)

    def calculate_loss(self, interaction):
        user = interaction[self.USER_ID]
        item = interaction[self.ITEM_ID]
        label = interaction[self.LABEL].float()
        output = self.forward(user, item)
        return self.focal_loss(output.squeeze(), label)

print("\n" + "="*60)
print("Running Focal Loss...")
print("="*60)

start_seed = len(results['focal'])
for i, seed in enumerate(SEEDS[start_seed:], start=start_seed):
    print(f"\nSeed {seed} ({i+1}/{len(SEEDS)})...")
    cfg = get_config()
    cfg['seed'] = seed
    config = Config(model='NeuMF', dataset=DATASET, config_dict=cfg)
    dataset_obj = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset_obj)
    model = NeuMFFocal(config, dataset_obj, gamma=2.0, alpha=0.25).to(device)
    trainer = Trainer(config, model)
    _, _ = trainer.fit(train_data, valid_data)
    test_result = trainer.evaluate(test_data)
    results['focal'].append(float(test_result['ndcg@10']))
    save_checkpoint()

print(f"\n✓ Focal complete: {np.mean(results['focal']):.4f} ± {np.std(results['focal']):.4f}")


Running Focal Loss...

Seed 0 (1/5)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

Seed 1 (2/5)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

Seed 2 (3/5)...


...

  → Checkpoint saved!

Seed 3 (4/5)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

Seed 4 (5/5)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

  → Checkpoint saved!

✓ Focal complete: 0.0638 ± 0.0041


In [ ]:
bce = np.array(results['bce'])
fl = np.array(results['focal'])

print("="*60)
print("FINAL RESULTS (Mean ± Std, 10 seeds)")
print("="*60)
print(f"BCE:   {np.mean(bce):.4f} ± {np.std(bce):.4f}")
print(f"Focal: {np.mean(fl):.4f} ± {np.std(fl):.4f}")
print(f"Change: {(np.mean(fl)-np.mean(bce))/np.mean(bce)*100:+.1f}%")

print("\n" + "="*60)
print("WILCOXON SIGNED-RANK TEST")
print("="*60)
stat, p = stats.wilcoxon(fl, bce)
d = np.mean(fl - bce) / np.std(fl - bce, ddof=1)
print(f"p-value: {p:.4f}")
print(f"Cohen's d: {d:.3f}")
print(f"Significant (α=0.05): {'YES ✓' if p < 0.05 else 'NO'}")

# Save final results
final_results = {
    'bce': {'values': results['bce'], 'mean': float(np.mean(bce)), 'std': float(np.std(bce))},
    'focal': {'values': results['focal'], 'mean': float(np.mean(fl)), 'std': float(np.std(fl))},
    'test': {'p_value': float(p), 'cohens_d': float(d), 'significant': p < 0.05}
}
with open('final_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)
print("\n✓ Saved to final_results.json")

FINAL RESULTS (Mean ± Std, 10 seeds)
BCE:   0.0580 ± 0.0030
Focal: 0.0638 ± 0.0041
Change: +10.0%

WILCOXON SIGNED-RANK TEST
p-value: 0.0625
Cohen's d: 2.632
Significant (α=0.05): NO


TypeError: Object of type bool_ is not JSON serializable